In [1]:
#@Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import numpy as np
import matplotlib.pyplot as plt

from IPython.display import clear_output
from retry import retry

from language_models.ollama_model import OllamaLanguageModel

from components.components import select_action, get_outcomes, multiple_choice, update_situation, mental_deliberation

In [2]:
system_message = ("This is an agent based model. "
  f"The goal of the LLM to to play characters in a game, and act as humanlike as possible. "
  "Ideally, human observers should not be able to tell the difference between the LLM and a human player. "
)

llm = OllamaLanguageModel(
"llama3", system_message=system_message, streaming=False
)

In [3]:
def generate_behaviours(model, personality, n_behaviours=40):
    """Generate a set of behaviours to sample from"""
    behaviours = []
    memories = prompt_string = " ".join(behaviours)



    request1 = (
        f"Think about a person with this {personality}. \n"
        "Come up with a behaviour that they might do that this person might find themselves in. \n"
        f"Here are other things that the person has already done: {memories}. \n"
        "As you are coming up with behaviours, have the behaviours be unique and different from each other. \n"
        "But generating a consistent person. \n"
        "Have you answers be less than 20 words but contain the relevant information. \n"
        "These should be concrete things that someone would have a memory of from the other person's perspective. \n"
        "Be sure to look through the memories that you have already created and not repeat a memory. \n"
        "If you generate a too similar memory, just make another one"
        "Look through the memories, and then make a memory of what would happen a few hours after the last memory. \n"
        "Don't tell me that you are creating the memory, just create the memory. \n"
        "Don't remind me of the prompt, just create the memory. \n"
        "Example: Saw they once get lost in a bookstore, trying to find exactly the right book, but they got lost in throught \n"
    )
    request2 = (
        f"Think about a person with a generic personality these should be mundane things. \n"
        "Come up with a behaviour that they might do that this person might find themselves in. \n"
        f"Here are other things that the person has already done: {memories}. \n"
        f"Create new ones like the mundane ones in this set. \n"
        "As you are coming up with behaviours, have the behaviours be unique and different from each other. \n"
        "Have these be as different as possible from the other behaviours but still mundane. \n"
        "Have you answers be less than 20 words but contain the relevant information. \n"
        "These should be concrete things that someone would have a memory of from the other person's perspective. \n"
        "Be sure to look through the memories that you have already created and not repeat a memory. \n"
        "If you generate a too similar memory, just make another one"
        "Look through the memories, and then make a memory of what would happen a few hours after the last memory. \n"
        "Don't tell me that you are creating the memory, just create the memory. \n"
        "Don't remind me of the prompt, just create the memory. \n"
        "Example: Saw they once get lost in a bookstore, trying to find exactly the right book, but they got lost in throught \n"
    )

    behaviours = []
    for i in range(n_behaviours):
        print(f"Generating behaviour {len(behaviours)+1}")
        memories = prompt_string = " ".join(behaviours)
        request = request2
        if i > 10:
            if i%2 == 0:
                request = request2
            else:
                request = request1
        output = model.sample_text(request)
        behaviours.append(output)
        print(output)
    return behaviours




In [48]:
behaviours = generate_behaviours(llm, "someone who is very confident, likes to takes risks, but can be careless with peoples feelings. Means well but can be a bit of a bull in a china shop", n_behaviours=40)

Generating behaviour 1
Spilled coffee on their shirt while rushing to catch a bus, and then laughed about it with a stranger who helped them clean up.
Generating behaviour 2
Spilled coffee on their shirt while rushing to grab a morning coffee before work.
Generating behaviour 3
Accidentally superglued their shoes to the floor during a casual morning stroll.
Generating behaviour 4
Spilled coffee on their shirt while rushing to catch a bus, then laughed about it with a stranger who offered them a napkin.
Generating behaviour 5
Accidentally superglued their shoes to the floor during a DIY project at home.
Generating behaviour 6
Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday".
Generating behaviour 7
Found a coupon for their favorite coffee shop on the floor and picked it up, feeling like they'd stumbled upon a treasure.
Generating behaviour 8
Spilled coffee on their shirt while rushing to catch the bus, and then laughed it off

In [49]:
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('all-MiniLM-L6-v2')


In [50]:
encoded_behaviours = []
for behaviour in behaviours:
    encoded_behaviours.append(st.encode(behaviour))

In [4]:
def get_affective_state(model, behaviours):
    """Get the affective perception of the agent"""
    # Get the affective state

    request = (
        f"Think about a person who has done these things {behaviours}. \n"
        "How do you feel about this person from the most negative a person could be being -1 to most positive a person could be being 1. \n"
        "If you feel neutral, put 0. \n"
        "Think about whether you would want to be friends with this person. \n"
        "Don't remind me of the prompt, just create the response. \n"
        "Only give me a number. \n"
        "Example: .3 \n"
    )
    output = model.sample_text(request)

    return float(output)

In [52]:
select_initial_memory = np.random.choice(len(encoded_behaviours))

from sklearn.metrics.pairwise import cosine_similarity


for i in range(20):
    similarity = cosine_similarity([encoded_behaviours[select_initial_memory]], [encoded_behaviours[i]])[0][0]
    if i == select_initial_memory:
        similarity = 0
    print(f"Memory {i+1}: {similarity}")
    print("\n")
    print(behaviours[select_initial_memory])
    print(behaviours[i])
    print("\n")




Memory 1: 0.570064902305603


Accidentally spilled coffee on a coworker's new shirt during a meeting, apologized profusely and offered to buy them lunch.
Spilled coffee on their shirt while rushing to catch a bus, and then laughed about it with a stranger who helped them clean up.


Memory 2: 0.6930854320526123


Accidentally spilled coffee on a coworker's new shirt during a meeting, apologized profusely and offered to buy them lunch.
Spilled coffee on their shirt while rushing to grab a morning coffee before work.


Memory 3: 0.3736720681190491


Accidentally spilled coffee on a coworker's new shirt during a meeting, apologized profusely and offered to buy them lunch.
Accidentally superglued their shoes to the floor during a casual morning stroll.


Memory 4: 0.611231803894043


Accidentally spilled coffee on a coworker's new shirt during a meeting, apologized profusely and offered to buy them lunch.
Spilled coffee on their shirt while rushing to catch a bus, then laughed about it wit

In [61]:
thought_record = []

for i in range(20):
    memory_ret = np.random.choice(len(encoded_behaviours))
    thought_record.append(behaviours[memory_ret])
    thought_record.append("\n")
    memories = " ".join(thought_record)
    print(memories)


    affective_state = get_affective_state(llm, memories)
    print(f"Turn {i}: {affective_state}")

Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday". 

Turn 0: 0.7
Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday". 
 Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday move". 

Turn 1: 0.7
Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday". 
 Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday move". 
 Spilled coffee on their shirt while rushing to grab a morning coffee before work. 

Turn 2: 0.7
Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday". 
 Spilled coffee on their shirt while rushing to a morning meeting, and then laughed it off as a "classic Monday move". 
 Spilled coffee on their shirt while rushing to grab a mornin

In [1]:
thought_record = []
for i in range(20):
    new_memory = input("New Memory:") 
    #thought_record.append(new_memory)
    #thought_record.append("\n")
    memories = " ".join(thought_record)
    print(memories)


    #affective_state = get_affective_state(llm, memories)
    #print(f"Turn {i}: {affective_state}")